In [1]:
import dnnlib
import legacy
import torch
import matplotlib.pyplot as plt
import numpy as np
import PIL.Image
import scipy.interpolate
import math
import random
import preprocessing.image_utils as itools
import cv2
import synthesis.synthesis as synth
from tqdm import tqdm
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
network_pkl = "../t-golden-fid14.19.pkl"
device = torch.device('cuda')
G = synth.load_generator(network_pkl, device)

Loading networks from "../t-golden-fid14.19.pkl"...
Network loaded.


In [3]:
synth.generate_video((3,3), 10, "../outputs/videos", G, device)

Seed count:  45.0
Setting up PyTorch plugin "bias_act_plugin"... Done.
Setting up PyTorch plugin "filtered_lrelu_plugin"... Done.


  0%|                                         | 1/10800 [00:06<18:24:45,  6.14s/it]


OSError: [Errno 32] Broken pipe

FFMPEG COMMAND:
/home/ubuntu/anaconda3/envs/stylegan3+meshrcnn/lib/python3.9/site-packages/imageio_ffmpeg/binaries/ffmpeg-linux64-v4.2.2 -y -f rawvideo -vcodec rawvideo -s 1024x1024 -pix_fmt rgb24 -r 60.00 -i - -an -vcodec libx264 -pix_fmt yuv420p -crf 25 -v warning /home/ubuntu/petals/outputs/videos/3x3-[285334,457868,83981,10584,117755,788355,935842,255928,200685,152450,188631,457424,579996,810044,390539,526363,83199,239592,685237,766493,180812,965796,181235,838613,754996,826661,155365,19259,519801,915128,907552,315811,397221,119497,169563,56274,530779,814739,919979,903122,552853,173077,925259,858463,605274].mp4

FFMPEG STDERR OUTPUT:
